# Import Stuff

In [206]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report, precision_recall_fscore_support, confusion_matrix
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
import xgboost as xgb

# Download Data

In [5]:
!kaggle competitions download -c spaceship-titanic

100%|█████████████████████████████████████████| 299k/299k [00:00<00:00, 480kB/s]
100%|█████████████████████████████████████████| 299k/299k [00:00<00:00, 479kB/s]


In [6]:
!unzip spaceship-titanic.zip

Archive:  spaceship-titanic.zip
  inflating: sample_submission.csv   
  inflating: test.csv                
  inflating: train.csv               


In [168]:
sample = pd.read_csv('/Users/Viet/Documents/Kaggle/Space_Titanic/kaggle_space_titanic/sample_submission.csv')
df = pd.read_csv('/Users/Viet/Documents/Kaggle/Space_Titanic/kaggle_space_titanic/train.csv')
df_test = pd.read_csv('/Users/Viet/Documents/Kaggle/Space_Titanic/kaggle_space_titanic/test.csv')

# Inspect the data

In [169]:
df.head()

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
0,0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,False
1,0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines,True
2,0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,False
3,0003_02,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,False
4,0004_01,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,Willy Santantines,True


In [172]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8693 entries, 0 to 8692
Data columns (total 18 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   PassengerId   8693 non-null   object 
 1   HomePlanet    8492 non-null   object 
 2   CryoSleep     8476 non-null   boolean
 3   Cabin         8494 non-null   object 
 4   Destination   8511 non-null   object 
 5   Age           8514 non-null   float64
 6   VIP           8490 non-null   boolean
 7   RoomService   8512 non-null   float64
 8   FoodCourt     8510 non-null   float64
 9   ShoppingMall  8485 non-null   float64
 10  Spa           8510 non-null   float64
 11  VRDeck        8505 non-null   float64
 12  Name          8493 non-null   object 
 13  Transported   8693 non-null   bool   
 14  group         8693 non-null   object 
 15  deck          8494 non-null   object 
 16  num           8494 non-null   object 
 17  side          8494 non-null   object 
dtypes: bool(1), boolean(2), floa

In [171]:
df['group'] = [str(x)[:4] for x in df['PassengerId']]
df['CryoSleep'] = df['CryoSleep'].astype('boolean')
df['VIP'] = df['VIP'].astype('boolean')
df['deck'], df['num'], df['side'] = zip(*[str(x).split('/') if x is not np.nan else [np.nan]*3 for x in df['Cabin']])

# Helpers

In [285]:
def model_statistics(func):
    def calc_model_statistics(*args, **kwargs):
        y_pred, verbose = func(*args, **kwargs)
        if verbose==True:
            print(classification_report(y_test, y_pred))
        precision, recall, f_score, support = precision_recall_fscore_support(y_test, y_pred, average='binary')
        print(f"""
              Precision:\t{precision:.2%}
              Recall:\t\t{recall:.2%}
              F-score:\t\t{f_score:.2%}
              """)

        return y_pred
    
    return calc_model_statistics

@model_statistics
def predict_and_print(model, X_test, verbose=False):
    y_pred = model.predict(X_test)
    
    return y_pred, verbose

# Logistic Regression

In [269]:
non_features = ['PassengerId', 'Name', 'Cabin']
X = df \
        .dropna() \
        .drop(['Transported'] + non_features, axis=1)
        
y = df \
        .dropna() \
        [['Transported']]

# make dummies
X_dummy = pd.get_dummies(X, dummy_na=True, drop_first=True)

X_train, X_test, y_train, y_test = train_test_split(X_dummy, y, test_size=0.2, random_state=42)

In [275]:
lr = LogisticRegression(random_state=42)
lr.fit(X_train, y_train)

/Users/Viet/opt/anaconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
/Users/Viet/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression(random_state=42)

In [287]:
y_pred = predict_and_print(lr, X_test, verbose=True)

              precision    recall  f1-score   support

       False       0.80      0.77      0.79       653
        True       0.79      0.81      0.80       669

    accuracy                           0.79      1322
   macro avg       0.79      0.79      0.79      1322
weighted avg       0.79      0.79      0.79      1322


              Precision:	78.70%
              Recall:		81.17%
              F-score:		79.91%
              
